In [3]:
!mkdir Case2
!ls


mkdir: cannot create directory ‘Case2’: File exists
Case18_seg1.mat			  debug			mtspec.py
Case1_seg10.mat			  fcn_shannon_entro.py	__pycache__
Case1_seg11.mat			  full_pipeline.py	runtime.py
Case1_seg12.mat			  full_pipeline.py.bac	spark-warehouse
Case1_seg1.mat			  get_features.py	temp
Case2				  get_features.pyc	TeraGen-1TB
Case5_seg1.mat			  MGH-TEST-01		test2.mat
compute_spectrogram_sunhaoqi.py   MGH-TEST-02
compute_spectrogram_sunhaoqi.pyc  MGH-TEST-7P


In [20]:
!pip install nitime

    100% |████████████████████████████████| 4.0MB 267kB/s 


In [19]:
!pip install joblib

    100% |████████████████████████████████| 184kB 2.2MB/s 


In [6]:
!pip install hdf5storage

    100% |████████████████████████████████| 61kB 4.0MB/s 


In [7]:
!pip install spectrum

    100% |████████████████████████████████| 102kB 5.4MB/s 
    100% |████████████████████████████████| 61kB 4.3MB/s 
  Running setup.py bdist_wheel for spectrum ... - \ done
  Stored in directory: /gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/.cache/pip/wheels/8c/57/db/35bde7acfd729f37526a6765ccbe4afe36fded493f1e65d675
  Running setup.py bdist_wheel for easydev ... - done
  Stored in directory: /gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/.cache/pip/wheels/56/f1/d6/b72aa4482beed23f727860bf69a8efee1c60836abf4ec2e779
Successfully built spectrum easydev


In [8]:
import os, sys, copy, time
from sys import stdin, stdout, stderr

# Test libraries
from scipy.signal import butter, filtfilt

# Loading
import hdf5storage
import h5py
import pylab

# numpy cleaning

from numpy.core.defchararray import lower
from scipy.linalg import schur

# tools
from scipy.io import loadmat
from scipy.special import gamma
from numpy import rint as fix
from scipy.linalg import schur
import numpy
from numpy import sqrt, prod, exp, log, dot, multiply, inf
from numpy.fft import fft2
from numpy.linalg import inv
from numpy.linalg import qr
from numpy import inf, empty

# spectral
from spectrum import pmtm
import spectrum

# features
from numpy import mean, std
from scipy.stats import kurtosis
from matplotlib.mlab import prctile

In [ ]:
# Test loading from file

from libhelpers import test_lib

test_lib()

In [ ]:
# Test mappartition
# TODO: distribute function to cluster

In [12]:
# Test calling hdf5 loader
import h5py
from io import StringIO
import requests
import json
import pandas as pd

objStorCred = {
  "auth_url": "https://identity.open.softlayer.com",
  "project": "object_storage_20ff227d_d66c_495a_a316_cd99e80a9e6f",
  "projectId": "a9fb4d478e3d40a8bbd54c5a2ecf25a3",
  "region": "dallas",
  "userId": "6a4cc8251c1940179a6cccc9098a15e0",
  "username": "admin_fd35793e7cf915d9a5a9c768b068029cf6d720b9",
  "password": "kDTcKA2H(3eo5.G0",
  "domainId": "a350f0fe7fb44571b29305706a12c95a",
  "domainName": "1334933",
  "role": "admin"
}

def getObjStoreFile(container, filename):
    url1 = 'https://identity.open.softlayer.com/v3/auth/tokens'
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': objStorCred['username'],'domain': {'id': objStorCred['domainId']},
            'password': objStorCred['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    #print resp1_body
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2, stream=True)
    if resp2.status_code == 200:
        if not os.path.exists(os.path.dirname(filename)):
            os.makedirs(os.path.dirname(filename))
        with open(filename, 'wb') as f:
            for chunk in resp2.iter_content(1024):
                f.write(chunk)
        return 'good'
    else:
        return 'bad'
    
print getObjStoreFile("MGH", "Case1/Case1_seg1.mat")

f = h5py.File("Case1_seg1.mat")
#getsizeof(f)



good


NameError: name 'getsizeof' is not defined

In [13]:
f.keys()

[u'Fs', u'channels', u'data', u'start_time']

In [16]:
mat=hdf5storage.loadmat('Case1_seg1.mat')
print(mat['Fs'],mat['Fs'].__class__)

print(mat['data'].shape)



(array([[ 255.99775]]), <type 'numpy.ndarray'>)
(32, 9999999)


In [46]:
!ls


Case18_seg1.mat  MGH-TEST-01  MGH-TEST-02  MGH-TEST-7P	spark-warehouse


In [ ]:
### Compute Spectrogram


from joblib import Parallel, delayed
import numpy as np
from scipy.signal import detrend
import nitime.algorithms as tsa


def compute_spec_each_seg(eeg_seg, NW, Fs):
    """
    Input:
    eeg_seg: numpy.array (point_num x channel_num)
    NW: Time-halfbandwidth product, 2 or 3 or 4 or ... #taper = 2NW-1
    Fs: sampling frequency in Hz
    
    Output:
    mt_pxx: the spectrum (freq_num x channel_num)
    freqs: frequencies (freq_num,)
    """
    point_num, channel_num = eeg_seg.shape
    nfft = max(1<<(point_num-1).bit_length(), point_num)
    freqs = np.arange(0, Fs, Fs*1.0/nfft)[:nfft//2+1]  # list of frequencies

    mt_pxx = np.zeros((len(freqs),channel_num))  # create the array to contain the spectrum

    eeg_seg = detrend(eeg_seg, axis=0)  # remove the overall trend of the signal
    for chi in range(channel_num):
        _, pxx, _ = tsa.multi_taper_psd(eeg_seg[:,chi], Fs=Fs, NW=NW, adaptive=True, jackknife=False, low_bias=True, NFFT=nfft)
        mt_pxx[:,chi] = pxx
        
    return mt_pxx, freqs


def mtspecgram_shq(eeg, movingwin, fpass, NW, Fs):
    """
    Input:
    eeg: numpy.array (signal_length x channel_num)
    movingwin: [window length, window step] in seconds
    fpass: [low, higher] in Hz
    NW: Time-halfbandwidth product, 2 or 3 or 4 or ... #taper = 2NW-1
    Fs: sampling frequency in Hz
    
    Output:
    spect: the spectrogram (window_num x freq_num x channel_num)
    stimes: window starting times (window_num,)
    sfreqs: frequencies (freq_num,)
    """
    signal_length, channel_num = eeg.shape
    window_length = int(round(movingwin[0]*Fs))
    window_step = int(round(movingwin[1]*Fs))
    
    window_start = np.arange(0,signal_length-window_length+1,window_step)  # starting point of each segment
    window_num = len(window_start)
    stimes = window_start*1./Fs
    
    n_jobs = 1  # number of cpus for parallel computing, -1 is all cpus
    verbose = True  # verbosity in parallel computing
    res = Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(compute_spec_each_seg)(eeg[window_start[wi]:window_start[wi]+window_length,:], NW, Fs) for wi in range(window_num))
        
    sfreqs = res[0][1]
    freq_good_ids = np.logical_and(sfreqs>=fpass[0], sfreqs<fpass[1])
    sfreqs = sfreqs[freq_good_ids]
    
    spect = np.array([rr[0][freq_good_ids] for rr in res])  # the spectrogram
        
    return spect, stimes, sfreqs
    
    
    
if __name__ == '__main__':
    #### 1.0 Loading data ####

    # - input -
    #dataPath = '/Data/'
    #print('dataPath',dataPath)
    
    # - output -
    #targetFolder = catstr('/Output/')
    
    fileName0='Case18_seg1.mat'
    
    print('Loading',fileName0)
    #fp=dataPath +fileName0
    fp=fileName0
    with load_mat(fp):
        matfile = load(fp)
        Fs=cellarray(matfile['Fs'])
        start_time = cellarray(matfile['start_time'])
        # channels = cellarray((51,1))
        channels=cellarray(matfile['channels'])
        
        eeg_data = matfile['data']

    print('data',eeg_data.T.shape,Fs,start_time,channels)
        
    #### 1.1 Compute spectrograms ####
    # just a toy example
#    eeg = np.random.rand(10000,6)
    
    spect, stimes, sfreqs = mtspecgram_shq(eeg_data.T, [2,2], [0.5, 20], 2, 200)
    print(spect.shape)
    print(stimes.shape)
    print(sfreqs.shape)
    



('Loading', 'Case18_seg1.mat')
open
close
('data', (9999999, 50), cellarray([[200.0]], dtype=object), cellarray([[u'07-29-2012 08:14:45']], dtype=object), cellarray([[u'C3   '],
       [u'C4   '],
       [u'O1   '],
       [u'O2   '],
       [u'A1   '],
       [u'A2   '],
       [u'Cz   '],
       [u'F3   '],
       [u'F4   '],
       [u'F7   '],
       [u'F8   '],
       [u'Fz   '],
       [u'Fp1  '],
       [u'Fp2  '],
       [u'Fpz  '],
       [u'P3   '],
       [u'P4   '],
       [u'Pz   '],
       [u'T3   '],
       [u'T4   '],
       [u'T5   '],
       [u'T6   '],
       [u'LOC  '],
       [u'ROC  '],
       [u'CHIN1'],
       [u'CHIN2'],
       [u'ECGL '],
       [u'ECGR '],
       [u'LAT1 '],
       [u'LAT2 '],
       [u'RAT1 '],
       [u'RAT2 '],
       [u'RFO1 '],
       [u'RFO2 '],
       [u'RFO3 '],
       [u'RFO4 '],
       [u'DIF5 '],
       [u'DIF6 '],
       [u'POS  '],
       [u'LFO1 '],
       [u'LFO2 '],
       [u'LFO3 '],
       [u'LFO4 '],
       [u'DC6  '],
     

/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/.local/lib/python2.7/site-packages/nitime/utils.py:551: RuntimeWarning: invalid value encountered in divide
  d_k /= eigvals[:, None]*sdf_iter[None, :] + bband_sup[:,None]
/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/.local/lib/python2.7/site-packages/numpy/lib/function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  else:


In [ ]:
import numpy as np
import sys
from numpy import sqrt, prod, exp, log, dot, multiply, inf
import hdf5storage

def load(fp):
    matfile=hdf5storage.loadmat(fp)
    # print('Loading from %s' % fp)
    return matfile

def size(a, b=0, nargout=1):
    """
    >>> size(zeros(3,3)) + 1
    matlabarray([[4, 4]])
    """
    s = np.asarray(a).shape
    if s is ():
        return 1 if b else (1,) * nargout
    # a is not a scalar
    try:
        if b:
            return s[b - 1]
        else:
            return matlabarray(s) if nargout <= 1 else s
    except IndexError:
        return 1

def arange(start, stop, step=1, **kwargs):
    """
    >>> a=arange(1,10) # 1:10
    >>> size(a)
    matlabarray([[ 1, 10]])
    """
    return matlabarray(np.arange(start,
                                 stop + 1,
                                 step,
                                 **kwargs).reshape(1, -1), **kwargs)


def cat(*args):
    return matlabarray(np.concatenate([matlabarray(a) for a in args], axis=1)).reshape(-1)


class end(object):
    def __add__(self, n):
        self.n = n
        return self

    def __sub__(self, n):
        self.n = -n
        return self

def isvector_or_scalar(a):
    """
    one-dimensional arrays having shape [N],
    row and column matrices having shape [1 N] and
    [N 1] correspondingly, and their generalizations
    having shape [1 1 ... N ... 1 1 1].
    Scalars have shape [1 1 ... 1].
    Empty arrays dont count
    """
    try:
        return a.size and a.ndim - a.shape.count(1) <= 1
    except:
        return False

def isvector(a):
    """
    one-dimensional arrays having shape [N],
    row and column matrices having shape [1 N] and
    [N 1] correspondingly, and their generalizations
    having shape [1 1 ... N ... 1 1 1]
    """
    try:
        return a.ndim - a.shape.count(1) == 1
    except:
        return False

class matlabarray(np.ndarray):
    """
    >>> matlabarray()
    matlabarray([], shape=(0, 0), dtype=float64)
    >>> matlabarray([arange(1,5), arange(1,5)])
    matlabarray([1, 2, 3, 4, 5, 1, 2, 3, 4, 5])
    >>> matlabarray(["hello","world"])
    matlabarray("helloworld")
    """

    def __new__(cls, a=[], dtype=None):
        # print(a.__class__)
        obj = np.array(a,
                       dtype=dtype,
                       copy=False,
                       order="F",
                       ndmin=2).view(cls).copy(order="F")
        if obj.size == 0:
            obj.shape = (0, 0)

        # self.obj=obj
        return obj

    # def __array_finalize__(self,obj):

    def __copy__(self):
        return np.ndarray.copy(self, order="F")

    def __iter__(self):
        """ must define iter or char won't work"""
        return np.asarray(self).__iter__()

    def compute_indices(self, index):
        if not isinstance(index, tuple):
            index = index,
        if len(index) != 1 and len(index) != self.ndim:
            raise IndexError
        indices = []
        for i, ix in enumerate(index):
            if ix.__class__ is end:
                indices.append(self.shape[i] - 1 + ix.n)
            elif ix.__class__ is slice:
                if self.size == 0 and ix.stop is None:
                    raise IndexError
                if len(index) == 1:
                    n = self.size
                else:
                    n = self.shape[i]
                indices.append(np.arange((ix.start or 1) - 1,
                                         ix.stop or n,
                                         ix.step or 1,
                                         dtype=int))
            else:
                try:
                    indices.append(int(ix) - 1)
                except:
                    indices.append(np.asarray(ix).astype("int32") - 1)
        if len(indices) == 2 and isvector(indices[0]) and isvector(indices[1]):
            indices[0].shape = (-1, 1)
            indices[1].shape = (-1,)
        return tuple(indices)

    def __getslice__(self, i, j):
        if i == 0 and j == sys.maxsize:
            return self.reshape(-1, 1, order="F")
        return self.__getitem__(slice(i, j))

    def __getitem__(self, index):
        return matlabarray(self.get(index))

    def get(self, index):
        # import pdb; pdb.set_trace()
        indices = self.compute_indices(index)
        if len(indices) == 1:
            return np.ndarray.__getitem__(self.reshape(-1, order="F"), indices)
        else:
            return np.ndarray.__getitem__(self, indices)

    def __setslice__(self, i, j, value):
        if i == 0 and j == sys.maxsize:
            index = slice(None, None)
        else:
            index = slice(i, j)
        self.__setitem__(index, value)

    def sizeof(self, ix):
        if isinstance(ix, int):
            n = ix + 1
        elif isinstance(ix, slice):
            n = ix.stop
        elif isinstance(ix, (list, np.ndarray)):
            n = max(ix) + 1
        else:
            assert 0, ix
        if not isinstance(n, int):
            raise IndexError
        return n

    def __setitem__(self, index, value):
        # import pdb; pdb.set_trace()
        indices = self.compute_indices(index)
        try:
            if len(indices) == 1:
                np.asarray(self).reshape(-1, order="F").__setitem__(indices, value)
            else:
                np.asarray(self).__setitem__(indices, value)
        except (ValueError, IndexError):
            # import pdb; pdb.set_trace()
            if not self.size:
                new_shape = [self.sizeof(s) for s in indices]
                self.resize(new_shape, refcheck=0)
                np.asarray(self).__setitem__(indices, value)
            elif len(indices) == 1:
                # One-dimensional resize is only implemented for
                # two cases:
                #
                # a. empty matrices having shape [0 0]. These
                #    matries may be resized to any shape.  A[B]=C
                #    where A=[], and B is specific -- A[1:10]=C
                #    rather than A[:]=C or A[1:end]=C
                if self.size and not isvector_or_scalar(self):
                    raise IndexError("One-dimensional resize "
                                     "works only on vectors, and "
                                     "row and column matrices")
                # One dimensional resize of scalars creates row matrices
                # ai = 3
                # a(4) = 1
                # 3 0 0 1
                n = self.sizeof(indices[0])  # zero-based
                if max(self.shape) == 1:
                    new_shape = list(self.shape)
                    new_shape[-1] = n
                else:
                    new_shape = [(1 if s == 1 else n) for s in self.shape]
                self.resize(new_shape, refcheck=0)
                np.asarray(self).reshape(-1, order="F").__setitem__(indices, value)
            else:
                new_shape = list(self.shape)
                if self.flags["C_CONTIGUOUS"]:
                    new_shape[0] = self.sizeof(indices[0])
                elif self.flags["F_CONTIGUOUS"]:
                    new_shape[-1] = self.sizeof(indices[-1])
                self.resize(new_shape, refcheck=0)
                np.asarray(self).__setitem__(indices, value)

    def __repr__(self):
        return self.__class__.__name__ + repr(np.asarray(self))[5:]

    def __str__(self):
        return str(np.asarray(self))

    def __add__(self, other):
        return matlabarray(np.asarray(self) + np.asarray(other))

    def __neg__(self):
        return matlabarray(np.asarray(self).__neg__())


class cellarray(matlabarray):
    """
    Cell array corresponds to matlab ``{}``


    """

    def __new__(cls, a=[]):
        """
        Create a cell array and initialize it with a.
        Without arguments, create an empty cell array.

        Parameters:
        a : list, ndarray, matlabarray, etc.

        >>> a=cellarray([123,"hello"])
        >>> print(a.shape)
        (1, 2)

        >>> print (a[1])
        123

        >>> print (a[2])
        hello

        >>> print(a)
        [[123,'hello']]

        >>> print('hello' in a)

        >>> print(strcmp(a,'hello'))
        True
        """
        obj = np.array(a,
                       dtype=object,
                       order="F",
                       ndmin=2).view(cls).copy(order="F")
        if obj.size == 0:
            obj.shape = (0, 0)
        return obj

    def __getitem__(self, index):
        return self.get(index)


    # def __str__(self):
    #        if self.ndim == 0:
    #            return ""
    #        if self.ndim == 1:
    #            return "".join(s for s in self)
    #        if self.ndim == 2:
    #            return "\n".join("".join(s) for s in self)
    #        raise NotImplementedError

def cell(*args):
    if len(args) == 1:
        args += args
    return cellarray(np.zeros(args, dtype=object, order="F"))


def copy(a):
    return matlabarray(np.asanyarray(a).copy(order="F"))


def find(a, n=None, d=None, nargout=1):
    if d:
        raise NotImplementedError

    # there is no promise that nonzero or flatnonzero
    # use or will use indexing of the argument without
    # converting it to array first.  So we use asarray
    # instead of asanyarray
    if nargout == 1:
        i = np.flatnonzero(np.asarray(a)).reshape(1, -1) + 1
        if n is not None:
            i = i.take(n)
        return matlabarray(i)
    if nargout == 2:
        i, j = np.nonzero(np.asarray(a))
        if n is not None:
            i = i.take(n)
            j = j.take(n)
        return (matlabarray((i + 1).reshape(-1, 1)),
                matlabarray((j + 1).reshape(-1, 1)))
    raise NotImplementedError


def disp(*args):
    print(args)
    
    
def catstr(*arg):
    """
    targetFolder='c:'
    fileName0='cdg'

    print(catstr(targetFolder, '/filtered_data_', fileName0))


    """

    return "".join(arg)


class Dict2Obj(object):
    """
    Turns a dictionary into a class
    """

    # ----------------------------------------------------------------------
    def __init__(self, dictionary):
        """Constructor"""
        for key in dictionary:
            setattr(self, key, dictionary[key])    

In [ ]:
from __future__ import division
import numpy as np
from spectrum import dpss
from scipy import signal
import scipy.io

"""
multitaper spectrogram

"""

def change_row_to_column(data):
    """
    row into column vector

    :param data:
    :return:
    """
    print(len(data.shape))
    if len(data.shape) > 1:
        N, C = data.shape
        if (N == 1):
            data = data.T
    else:
        print(data.shape[0], 1)

        data = data.reshape((data.shape[0], 1))
    print(data.shape)
    return data


def nextpow2(n):
    if np.any(n < 0):
        raise ValueError("n should be > 0")

    if np.isscalar(n):
        f, p = np.frexp(n)
        if f == 0.5:
            return p - 1
        elif np.isfinite(f):
            return p
        else:
            return f
    else:
        f, p = np.frexp(n)
        res = f
        bet = np.isfinite(f)
        exa = (f == 0.5)
        res[bet] = p[bet]
        res[exa] = p[exa] - 1
        return res


def getfgrid(Fs, nfft, fpass):
    """
    get freq grid given fft
    """
    df = Fs / nfft
    f = np.arange(0, Fs, df)
    l = len(f)
    if f[l - 1] != Fs:
        f = np.append(f, Fs)
    f = f[0:nfft]
    if len(fpass) != 1:
        findx = (np.where((f >= fpass[0]) & (f <= fpass[1])))
    else:
        min_index, min_value = min(enumerate(f - fpass[2]), key=np.operator.itemgetter(1))
        f = f[min_index]
        findx = min_index
    fout = f[findx]
    return fout, findx[0]


def dpsschk(tapers, N, Fs):
    [tapers, eigs] = dpss(N, tapers[0], tapers[1])
    tapers = np.multiply(tapers, np.sqrt(Fs))
    return tapers, eigs

def mtfftc(data, tapers, nfft, Fs):
    data = change_row_to_column(data)
    NC, C = data.shape
    NK, K = tapers.shape
    if NK != NC:
        print("length of tapers is not compatible with length of data!!")
    # tapers=tapers[:,:,np.ones()]
    # data=data[:,:,ones()]
    # data=np.transpose(data, (0, 2, 1))
    data_mat = np.tile(data, (1, K))  # to create the matrix which has n rows X  n1cols
    data_proj = np.multiply(data_mat, tapers)
    J = np.fft.fft(data_proj, n=nfft, axis=0) / Fs
    return J


def mt_spectrogram(data, *args):
    print('calculate spectr')

    if len(args) == 1:
        Fs = args[0]
    elif len(args) == 2:
        Fs = args[0]
        params = args[1]

    if 'params' in locals():
        print('params is defined')
        pad1 = params.pad1
        fpass = params.fpass
        trialave = params.trialave
        err = params.err
        movingwin = params.movingwin
        tapers = params.tapers
    else:
        print('params is not defined!!')
        pad1 = 0
        fpass = [0, 55]
        trialave = 0
        err = 0
        movingwin = [4, 1]
        tapers = [3, 5]

        Fs = 200
        fpass = [0.5, 20]
        trialve = 0
        err = 0
        movingwin = [2, 2]
        tapers = [2, 3]

    data = change_row_to_column(data)
    [N, Ch] = data.shape
    Nwin = round(Fs * movingwin[0])
    Nstep = round(movingwin[1] * Fs)
    nextPow2v = nextpow2(Nwin)
    nfft = max(pow(2, nextPow2v + pad1), Nwin)
    [sfreqs, findx] = getfgrid(Fs, nfft, fpass)
    Nf = len(sfreqs)
    tapersMod = dpsschk(tapers, N, Fs)

    endV = N - Nwin + 1
    winstart = np.arange(0, endV, Nstep)
    if winstart[len(winstart) - 1] != endV:
        winstart = np.append(winstart, endV)

    nw = len(winstart) - 1

    if trialave == 0:
        S = np.zeros((nw, Nf))
    else:
        S = np.zeros((nw, Nf, Ch))

    """ tapers are calculated only in for first two loopsb """
    for ii in range(0, 2):
        start = winstart[ii]
        end = winstart[ii] + Nwin
        indx = np.arange(start, end)
        indx = indx.astype(int)

        datawin = signal.detrend(data[indx], type == "constant")
        # datawin = signal.detrend(data[indx])
        datawin = change_row_to_column(datawin)
        N = datawin.shape[0]
        taps = dpsschk(tapers, N, Fs)
        J = mtfftc(datawin, taps[0], nfft, Fs)
        J = J[findx, :]
        s = np.mean((np.multiply(np.conj(J), J)), axis=1).squeeze()
        if trialave == 1:
            s = (np.mean(s, axis=1).squeeze())
        S[ii, :] = s

    for ii in range(2, nw - 1):
        start = winstart[ii]
        end = winstart[ii] + Nwin
        indx = np.arange(start, end)
        indx = indx.astype(int)
        datawin = signal.detrend(data[indx], type == "constant")
        # datawin = signal.detrend(data[indx])
        datawin = change_row_to_column(datawin)
        N = datawin.shape[0]
        taps = dpsschk(tapers, N, Fs)
        J = mtfftc(datawin, taps[0], nfft, Fs)
        J = J[findx, :]
        s = np.mean((np.multiply(np.conj(J), J)), axis=1).squeeze()
        if trialave == 1:
            s = (np.mean(s, axis=1).squeeze())
        S[ii, :] = s

    spect = S.squeeze()  # final spectrogram matrix

    winmid = winstart[:-1] + np.round(Nwin / 2)
    stimes = winmid / Fs  # contains the time stamps

    return spect, stimes, sfreqs



def save(fp, d):
    # print('saveing',fp)
    # hdf5storage.savemat(fp,d)
    fp = fp + '.mat'
    print('Saving to %s' % fp, '| variables;', *['%s' % e for e in d.keys()])

    if os.path.exists(fp):
        os.remove(fp)
    for e in d.keys():
        obj = d[e]
        print('key', e, 'shape', obj.shape, obj.dtype)
        print(obj)

        def apply_f(a, f):
            # if isinstance(a, list):
            print(str(type(a)))
            if 'cellarray' in str(type(a)):
                print('cell')
                return map(lambda t: apply_f(t, f), a)
            else:
                print('other')
                return f(a)

        out_obj = list(apply_f(obj, lambda e: e.copy()))

        hdf5storage.write(data=out_obj, path='/%s' % e, filename=fp,
                          store_python_metadata=False, matlab_compatible=True)

In [ ]:
from joblib import Parallel, delayed
import numpy as np
from numpy.core.defchararray import lower
from scipy.signal import detrend
import nitime.algorithms as tsa

#from tsne.func.matlablib.runtime import cell, arange, size, find, matlabarray
#from tsne.func.matlablib.runtime import cellarray


def step1_load_subtract(fileName0):
    fp = fileName0
    with load_mat(fp):
        matfile = load(fp)
        Fs = cellarray(matfile['Fs'])
        start_time = cellarray(matfile['start_time'])
        # channels = cellarray((51,1))
        channels = cellarray(matfile['channels'])
        data = cellarray(matfile['data'])

    
    # TODO: Resample to 200 Hz!!! %
    # data = np.resample(data, 200, Fs)
    # data = data.T
    # Fs = 200


    print('targetFolder', targetFolder)
    
    def strrep_s(a, x, y):
        return a.replace(x, y)

    strrep = np.vectorize(strrep_s)

    print('channels', channels)
    print(size(channels, 1))
    # df_s0=pd.DataFrame()
    labels = copy(channels)

    for i in arange(1, size(channels, 1)).reshape(-1):
        #     print(i)
        x = strrep(channels[i, :], ' ', '')
        labels[i] = lower(x)


    disp('load 0')
    disp('compute 1')
    # pd=copy(pwd)
    movingwin = matlabarray(cat(2, 2))

    # dictobj
    params = Dict2Obj({'default': 1})

    params.pad = copy(0)
    params.fpass = copy(cat(0.5, 20))
    params.err = copy(0)
    params.trialave = copy(0)
    params.tapers = copy(cat(2, 3))
    params.Fs = copy(200)


    COI = cell(4, )
    COI[1] = cellarray([['fp1', 'f7'], ['f7', 't3'], ['t3', 't5'], ['t5', 'o1']])
    COI[2] = cellarray([['fp1', 'f3'], ['f3', 'c3'], ['c3', 'p3'], ['p3', 'o1']])
    COI[3] = cellarray([['fp2', 'f4'], ['f4', 'c4'], ['c4', 'p4'], ['p4', 'o2']])
    COI[4] = cellarray([['fp2', 'f8'], ['f8', 't4'], ['t4', 't6'], ['t6', 'o2']])

    print('COI', COI)

    ROInickname = cellarray(['LL', 'LP', 'RP', 'RL'])
    print(ROInickname)

    # test
    def strcmp_s(x, y):
        return x == y
    strcmp = np.vectorize(strcmp_s)

    dataB = cell(4, 4)
    for kk in arange(1, 4).reshape(-1):
        coi = COI[kk]

        for k in arange(1, size(coi, 1)).reshape(-1):
            print('kk', kk, 'k', k, coi[k, 1], coi[k, 2])

            c1 = find(strcmp(labels, coi[k, 1]))
            c2 = find(strcmp(labels, coi[k, 2]))
            c1 = c1.item()
            c2 = c2.item()
            print('items', c1, c2)
            dataB[kk, k] = np.array((data[c1, :] - data[c2, :]).tolist())

    d = {}
    d['dataB'] = dataB
    outp = catstr(targetFolder, '/strcmp_', fileName0)
    save(outp, d)
    
    
    # compute spectrogram for each channel
    spect = cell(4, 4)
    disp('compute 2', 'compute spectrogram for each channel')
    print('params', params.Fs, movingwin)

    for kk in arange(1, 4).reshape(-1):
        for k in arange(1, 4).reshape(-1):
            print('kk', kk, 'k', k)
            s = dataB[kk, k]
            params.Fs = copy(Fs).item()

            S, stimes, sfreqs = mt_spectrogram(s, Fs)

            spect[kk, k] = np.array(S.T)
            # Average over this region
    d = {}
    d['dataB'] = dataB
    outp = catstr(targetFolder, '/compute2_', fileName0)
    save(outp, d)
    
    print(spect.shape)
    

if __name__ == '__main__':
    #### 1.0 Loading data ####

    # - input -
    #dataPath = '/Data/'
    #print('dataPath',dataPath)
    
    # - output -
    targetFolder = catstr('./')
    
    fileName0='Case18_seg1.mat'
    
    print('Loading',fileName0)
    
    step1_load_subtract(fileName0)

In [ ]:
# get_features function
def fcn_shannon_entro(x0=None,*args,**kwargs):
    M,N=size(x0,nargout=2)
    # y=zeros(1,N)
    # y=zeros((1,N))
    x=np.array(x0[:],dtype=complex)
    y=matlabarray(np.zeros((1,N),dtype=complex))
    for l in arange(1,N).reshape(-1):
        # sum1=sum(multiply(x[:,l],log(x[:,l])))
        sum1 = sum(multiply(x[:], log(x[:])))
        sum1=dot(-1,sum1)
        y[1,l]=sum1

    return y


def bandpower(Pxx, f, frange, opt='psd'):
    """ integrate the power spectral density between fmin and fmax
        using the trapezoidal method
    """
    (fmin, fmax) = frange
    ind_min = scipy.argmax(f > fmin) - 1

    ind_max = scipy.argmax(f > fmax) + 1
    print(ind_min, ind_max)
    return Pxx[ind_min: ind_max].sum(axis=0) * np.diff(f)[0]


def get_features(data=None, i=None, window_length=None, spect=None, sfreqs=None, spectwindow_length=None, Fs=None,
                 *args, **kwargs):
    # time features: line length, kurtosis, shannon entropy and nonlinear energy.
    # spectral features: delta, theta, alpha and beta bandpower. delta to theta
    # ratio, theta to alpha ratio and delta to alpha ratio. Kurtosis of delta,
    # theta, alpha and beta bandpower.

    print("compute", i)

    # window_length = [14 10 6 2]*Fs; # the different windows for each segment

    t = dot((i - 1), window_length[4]) + dot(0.5, window_length[1])

    line1avg = 0
    line2avg = 0
    line3avg = 0
    line4avg = 0

    kurt1avg = 0
    kurt2avg = 0
    kurt3avg = 0
    kurt4avg = 0

    shan1avg = 0
    shan2avg = 0
    shan3avg = 0
    shan4avg = 0

    psy1meanavg = 0
    psy2meanavg = 0
    psy3meanavg = 0
    psy4meanavg = 0

    psy1stdavg = 0
    psy2stdavg = 0
    psy3stdavg = 0
    psy4stdavg = 0

    for k in arange(1, 4).reshape(-1):
        # win1=data[k][arange(t - dot(0.5,window_length[1]) + 1,t + dot(0.5,window_length[1]))]
        # win2=data[k][arange(t - dot(0.5,window_length[2]) + 1,t + dot(0.5,window_length[2]))]
        # win3=data[k][arange(t - dot(0.5,window_length[3]) + 1,t + dot(0.5,window_length[3]))]
        # win4=data[k][arange(t - dot(0.5,window_length[4]) + 1,t + dot(0.5,window_length[4]))]
        # win1=matlabarray([])
        win1o = data[k][t - dot(0.5, window_length[1]):t + dot(0.5, window_length[1])]
        win2o = data[k][t - dot(0.5, window_length[2]):t + dot(0.5, window_length[2])]
        win3o = data[k][t - dot(0.5, window_length[3]):t + dot(0.5, window_length[3])]
        win4o = data[k][t - dot(0.5, window_length[4]):t + dot(0.5, window_length[4])]

        win1 = matlabarray(data[k][t - dot(0.5, window_length[1]):t + dot(0.5, window_length[1])])
        win2 = matlabarray(data[k][t - dot(0.5, window_length[2]):t + dot(0.5, window_length[2])])
        win3 = matlabarray(data[k][t - dot(0.5, window_length[3]):t + dot(0.5, window_length[3])])
        win4 = matlabarray(data[k][t - dot(0.5, window_length[4]):t + dot(0.5, window_length[4])])

        # time features
        line1 = 0
        line2 = 0
        line3 = 0
        line4 = 0

        for j in arange(1, window_length[1] - 1).reshape(-1):
            line1 = line1 + abs(win1[j + 1] - win1[j])

        line1 = line1 / (window_length[1] - 1)

        for j in arange(1, window_length[2] - 1).reshape(-1):
            line2 = line2 + abs(win2[j + 1] - win2[j])

        line2 = line2 / (window_length[2] - 1)

        for j in arange(1, window_length[3] - 1).reshape(-1):
            line3 = line3 + abs(win3[j + 1] - win3[j])

        line3 = line3 / (window_length[3] - 1)

        for j in arange(1, window_length[4] - 1).reshape(-1):
            line4 = line4 + abs(win4[j + 1] - win4[j])

        line4 = line4 / (window_length[4] - 1)

        kurt1 = kurtosis(win1o, 0)
        kurt2 = kurtosis(win2o, 0)
        kurt3 = kurtosis(win3o, 0)
        kurt4 = kurtosis(win4o, 0)

        #     samp_en1 = fcn_SampEn(2, 0.2*std(win1), win1); # sample entropy
        #     samp_en2 = fcn_SampEn(2, 0.2*std(win2), win2);
        #     samp_en3 = fcn_SampEn(2, 0.2*std(win3), win3);
        #     samp_en4 = fcn_SampEn(2, 0.2*std(win4), win4);
        shan1 = abs(fcn_shannon_entro(win1.T))
        shan2 = abs(fcn_shannon_entro(win2.T))
        shan3 = abs(fcn_shannon_entro(win3.T))
        shan4 = abs(fcn_shannon_entro(win4.T))

        # psy1=zeros(1,length(win1) - 3)
        psy1 = matlabarray(np.zeros((1, length(win1) - 3)))

        for n in arange(4, length(win1)).reshape(-1):
            psy1[n - 3] = dot(win1[n - 1], win1[n - 2]) - dot(win1[n], win1[n - 3])

        psy1mean = mean(abs(psy1))
        psy1std = std(psy1, ddof=1)

        # psy2=zeros(1,length(win2) - 3)
        psy2 = matlabarray(np.zeros((1, length(win2) - 3)))

        for n in arange(4, length(win2)).reshape(-1):
            psy2[n - 3] = dot(win2[n - 1], win2[n - 2]) - dot(win2[n], win2[n - 3])

        psy2mean = mean(abs(psy2))
        psy2std = std(psy2, ddof=1)

        # psy3=zeros(1,length(win3) - 3)
        psy3 = matlabarray(np.zeros((1, length(win3) - 3)))

        for n in arange(4, length(win3)).reshape(-1):
            psy3[n - 3] = dot(win3[n - 1], win3[n - 2]) - dot(win3[n], win3[n - 3])

        psy3mean = mean(abs(psy3))
        psy3std = std(psy3, ddof=1)

        # psy4=zeros(1,length(win4) - 3)
        psy4 = matlabarray(np.zeros((1, length(win4) - 3)))

        for n in arange(4, length(win4)).reshape(-1):
            psy4[n - 3] = dot(win4[n - 1], win4[n - 2]) - dot(win4[n], win4[n - 3])

        psy4mean = mean(abs(psy4))
        psy4std = std(psy4, ddof=1)

        line1avg = line1avg + line1
        line2avg = line2avg + line2
        line3avg = line3avg + line3
        line4avg = line4avg + line4

        kurt1avg = kurt1avg + kurt1
        kurt2avg = kurt2avg + kurt2
        kurt3avg = kurt3avg + kurt3
        kurt4avg = kurt4avg + kurt4

        shan1avg = shan1avg + shan1
        shan2avg = shan2avg + shan2
        shan3avg = shan3avg + shan3
        shan4avg = shan4avg + shan4

        psy1meanavg = psy1meanavg + psy1mean
        psy2meanavg = psy2meanavg + psy2mean
        psy3meanavg = psy3meanavg + psy3mean
        psy4meanavg = psy4meanavg + psy4mean

        psy1stdavg = psy1stdavg + psy1std
        psy2stdavg = psy2stdavg + psy2std
        psy3stdavg = psy3stdavg + psy3std
        psy4stdavg = psy4stdavg + psy4std

    line1avg = line1avg / 4
    line2avg = line2avg / 4
    line3avg = line3avg / 4
    line4avg = line4avg / 4

    kurt1avg = kurt1avg / 4
    kurt2avg = kurt2avg / 4
    kurt3avg = kurt3avg / 4
    kurt4avg = kurt4avg / 4

    shan1avg = shan1avg / 4
    shan2avg = shan2avg / 4
    shan3avg = shan3avg / 4
    shan4avg = shan4avg / 4

    psy1meanavg = psy1meanavg / 4
    psy2meanavg = psy2meanavg / 4
    psy3meanavg = psy3meanavg / 4
    psy4meanavg = psy4meanavg / 4

    psy1stdavg = psy1stdavg / 4
    psy2stdavg = psy2stdavg / 4
    psy3stdavg = psy3stdavg / 4
    psy4stdavg = psy4stdavg / 4

    # spectral features
    #### t1 = ceil(t/Fs/2);
    tt1 = ceil(t / Fs / 2 - dot(0.5, spectwindow_length))
    tt2 = ceil(t / Fs / 2 + dot(0.5, spectwindow_length))
    tt1 = tt1.astype(int)
    tt2 = tt2.astype(int)

    # edit: matrix select---
    # spectwin1=spect[:,tt1[1] + 1:tt2[1]]
    # spectwin2=spect[:,tt1[2] + 1:tt2[2]]
    # spectwin3=spect[:,tt1[3] + 1:tt2[3]]
    # spectwin4=spect[:,tt1[4] + 1:tt2[4]]
    spectwin1 = spect[:, tt1[1]:tt2[1]]
    spectwin2 = spect[:, tt1[2]:tt2[2]]
    spectwin3 = spect[:, tt1[3]:tt2[3]]
    spectwin4 = spect[:, tt1[4]:tt2[4]]

    eps = 0.0
    delta1 = bandpower(spectwin1, sfreqs, cat(1, 4), 'psd')
    theta1 = bandpower(spectwin1, sfreqs, cat(4, 8), 'psd')
    alpha1 = bandpower(spectwin1, sfreqs, cat(8, 12), 'psd')
    beta1 = bandpower(spectwin1, sfreqs, cat(12, 18), 'psd')
    total1 = bandpower(spectwin1, sfreqs, cat(1, sfreqs[len(sfreqs) - 2]), 'psd') + eps
    # total1 = bandpower(spectwin1, sfreqs, cat(1, sfreqs[end()]), 'psd')

    delta1_rat = delta1 / total1
    theta1_rat = theta1 / total1
    alpha1_rat = alpha1 / total1
    beta1_rat = beta1 / total1

    delta_theta1 = delta1 / (theta1 + eps)
    delta_alpha1 = delta1 / (alpha1 + eps)
    theta_alpha1 = theta1 / (alpha1 + eps)

    kurtdelta1 = kurtosis(delta1, 0)
    kurttheta1 = kurtosis(theta1, 0)
    kurtalpha1 = kurtosis(alpha1, 0)
    kurtbeta1 = kurtosis(beta1, 0)

    delta1_mean = mean(delta1_rat)
    theta1_mean = mean(theta1_rat)
    alpha1_mean = mean(alpha1_rat)
    beta1_mean = mean(beta1_rat)

    delta1_min = min(delta1_rat)
    theta1_min = min(theta1_rat)
    alpha1_min = min(alpha1_rat)
    beta1_min = min(beta1_rat)

    delta1_std = std(delta1_rat)
    theta1_std = std(theta1_rat)
    alpha1_std = std(alpha1_rat)
    beta1_std = std(beta1_rat)

    delta1_prct = prctile(delta1_rat, 95)
    theta1_prct = prctile(theta1_rat, 95)
    alpha1_prct = prctile(alpha1_rat, 95)
    beta1_prct = prctile(beta1_rat, 95)

    delthe1_mean = mean(delta_theta1)
    delalph1_mean = mean(delta_alpha1)
    thealph1_mean = mean(theta_alpha1)

    delthe1_min = min(delta_theta1)
    delalph1_min = min(delta_alpha1)
    thealph1_min = min(theta_alpha1)

    delthe1_std = std(delta_theta1)
    delalph1_std = std(delta_alpha1)
    thealph1_std = std(theta_alpha1)

    delthe1_prct = prctile(delta_theta1, 95)
    delalph1_prct = prctile(delta_alpha1, 95)
    thealph1_prct = prctile(theta_alpha1, 95)

    delta2 = bandpower(spectwin2, sfreqs, cat(sfreqs[1], 4), 'psd')
    theta2 = bandpower(spectwin2, sfreqs, cat(4, 8), 'psd')
    alpha2 = bandpower(spectwin2, sfreqs, cat(8, 12), 'psd')
    beta2 = bandpower(spectwin2, sfreqs, cat(12, 18), 'psd')
    total2 = bandpower(spectwin2, sfreqs, cat(1, sfreqs[len(sfreqs) - 2]), 'psd') + eps
    # total2=bandpower(spectwin2,sfreqs,'psd') + eps
    # total2=beta2

    delta2_rat = delta2 / total2
    theta2_rat = theta2 / total2
    alpha2_rat = alpha2 / total2
    beta2_rat = beta2 / total2

    delta_theta2 = delta2 / (theta2 + eps)
    delta_alpha2 = delta2 / (alpha2 + eps)
    theta_alpha2 = theta2 / (alpha2 + eps)

    kurtdelta2 = kurtosis(delta2, 0)
    kurttheta2 = kurtosis(theta2, 0)
    kurtalpha2 = kurtosis(alpha2, 0)
    kurtbeta2 = kurtosis(beta2, 0)

    delta2_mean = mean(delta2_rat)
    theta2_mean = mean(theta2_rat)
    alpha2_mean = mean(alpha2_rat)
    beta2_mean = mean(beta2_rat)

    delta2_min = min(delta2_rat)
    theta2_min = min(theta2_rat)
    alpha2_min = min(alpha2_rat)
    beta2_min = min(beta2_rat)

    delta2_std = std(delta2_rat)
    theta2_std = std(theta2_rat)
    alpha2_std = std(alpha2_rat)
    beta2_std = std(beta2_rat)

    delta2_prct = prctile(delta2_rat, 95)
    theta2_prct = prctile(theta2_rat, 95)
    alpha2_prct = prctile(alpha2_rat, 95)
    beta2_prct = prctile(beta2_rat, 95)

    delthe2_mean = mean(delta_theta2)
    delalph2_mean = mean(delta_alpha2)
    thealph2_mean = mean(theta_alpha2)

    delthe2_min = min(delta_theta2)
    delalph2_min = min(delta_alpha2)
    thealph2_min = min(theta_alpha2)

    delthe2_std = std(delta_theta2)
    delalph2_std = std(delta_alpha2)
    thealph2_std = std(theta_alpha2)

    delthe2_prct = prctile(delta_theta2, 95)
    delalph2_prct = prctile(delta_alpha2, 95)
    thealph2_prct = prctile(theta_alpha2, 95)

    delta3 = bandpower(spectwin3, sfreqs, cat(sfreqs[1], 4), 'psd')
    theta3 = bandpower(spectwin3, sfreqs, cat(4, 8), 'psd')
    alpha3 = bandpower(spectwin3, sfreqs, cat(8, 12), 'psd')
    beta3 = bandpower(spectwin3, sfreqs, cat(12, 18), 'psd')
    # total3=bandpower(spectwin3,sfreqs,'psd') + eps
    total3 = bandpower(spectwin3, sfreqs, cat(1, sfreqs[len(sfreqs) - 2]), 'psd') + eps

    delta3_rat = delta3 / total3
    theta3_rat = theta3 / total3
    alpha3_rat = alpha3 / total3
    beta3_rat = beta3 / total3

    delta_theta3 = delta3 / (theta3 + eps)
    delta_alpha3 = delta3 / (alpha3 + eps)
    theta_alpha3 = theta3 / (alpha3 + eps)

    kurtdelta3 = kurtosis(delta3, 0)
    kurttheta3 = kurtosis(theta3, 0)
    kurtalpha3 = kurtosis(alpha3, 0)
    kurtbeta3 = kurtosis(beta3, 0)

    delta3_mean = mean(delta3_rat)
    theta3_mean = mean(theta3_rat)
    alpha3_mean = mean(alpha3_rat)
    beta3_mean = mean(beta3_rat)

    delta3_min = min(delta3_rat)
    theta3_min = min(theta3_rat)
    alpha3_min = min(alpha3_rat)
    beta3_min = min(beta3_rat)

    delta3_std = std(delta3_rat)
    theta3_std = std(theta3_rat)
    alpha3_std = std(alpha3_rat)
    beta3_std = std(beta3_rat)

    delta3_prct = prctile(delta3_rat, 95)
    theta3_prct = prctile(theta3_rat, 95)
    alpha3_prct = prctile(alpha3_rat, 95)
    beta3_prct = prctile(beta3_rat, 95)

    delthe3_mean = mean(delta_theta3)
    delalph3_mean = mean(delta_alpha3)
    thealph3_mean = mean(theta_alpha3)

    delthe3_min = min(delta_theta3)
    delalph3_min = min(delta_alpha3)
    thealph3_min = min(theta_alpha3)

    delthe3_std = std(delta_theta3)
    delalph3_std = std(delta_alpha3)
    thealph3_std = std(theta_alpha3)

    delthe3_prct = prctile(delta_theta3, 95)
    delalph3_prct = prctile(delta_alpha3, 95)
    thealph3_prct = prctile(theta_alpha3, 95)

    delta4 = bandpower(spectwin4, sfreqs, cat(sfreqs[1], 4), 'psd')
    theta4 = bandpower(spectwin4, sfreqs, cat(4, 8), 'psd')
    alpha4 = bandpower(spectwin4, sfreqs, cat(8, 12), 'psd')
    beta4 = bandpower(spectwin4, sfreqs, cat(12, 18), 'psd')
    # total4=bandpower(spectwin4,sfreqs,'psd') + eps
    total4 = bandpower(spectwin4, sfreqs, cat(1, sfreqs[len(sfreqs) - 2]), 'psd') + eps

    delta4_rat = delta4 / total4
    theta4_rat = theta4 / total4
    alpha4_rat = alpha4 / total4
    beta4_rat = beta4 / total4

    delta_theta4 = delta4 / (theta4 + eps)
    delta_alpha4 = delta4 / (alpha4 + eps)
    theta_alpha4 = theta4 / (alpha4 + eps)

    # kurtdelta4 = kurtosis(delta4);
    # kurttheta4 = kurtosis(theta4);
    # kurtalpha4 = kurtosis(alpha4);
    # kurtbeta4 = kurtosis(beta4);
    delta4_mean = mean(delta4_rat)
    theta4_mean = mean(theta4_rat)
    alpha4_mean = mean(alpha4_rat)
    beta4_mean = mean(beta4_rat)

    delta4_min = min(delta4_rat)
    theta4_min = min(theta4_rat)
    alpha4_min = min(alpha4_rat)
    beta4_min = min(beta4_rat)

    delta4_std = std(delta4_rat)
    theta4_std = std(theta4_rat)
    alpha4_std = std(alpha4_rat)
    beta4_std = std(beta4_rat)

    delta4_prct = prctile(delta4_rat, 95)
    theta4_prct = prctile(theta4_rat, 95)
    alpha4_prct = prctile(alpha4_rat, 95)
    beta4_prct = prctile(beta4_rat, 95)

    delthe4_mean = mean(delta_theta4)
    delalph4_mean = mean(delta_alpha4)
    thealph4_mean = mean(theta_alpha4)

    delthe4_min = min(delta_theta4)
    delalph4_min = min(delta_alpha4)
    thealph4_min = min(theta_alpha4)

    delthe4_std = std(delta_theta4)
    delalph4_std = std(delta_alpha4)
    thealph4_std = std(theta_alpha4)

    delthe4_prct = prctile(delta_theta4, 95)
    delalph4_prct = prctile(delta_alpha4, 95)
    thealph4_prct = prctile(theta_alpha4, 95)

    line1avg = line1avg.item()
    line2avg = line2avg.item()
    line3avg = line3avg.item()
    line4avg = line4avg.item()
    features1 = matlabarray(
        cat([line1avg], [line2avg], [line3avg], [line4avg], [kurt1avg], [kurt2avg], [kurt3avg], [kurt4avg]))
    features1a = matlabarray(
        cat([shan1avg], [shan2avg], [shan3avg], [shan4avg], [psy1meanavg], [psy2meanavg], [psy3meanavg], [psy4meanavg],
            [psy1stdavg], [psy2stdavg], [psy3stdavg], [psy4stdavg]))
    features1aa = matlabarray(
        cat([delta1_mean], [delta2_mean], [delta3_mean], [delta4_mean], [theta1_mean], [theta2_mean], [theta3_mean],
            [theta4_mean], [alpha1_mean], [alpha2_mean], [alpha3_mean], [alpha4_mean]))
    features1b = matlabarray(
        cat([beta1_mean], [beta2_mean], [beta3_mean], [beta4_mean], [delta1_min], [delta2_min], [delta3_min],
            [theta1_min], [theta2_min], [theta3_min], [alpha1_min], [alpha2_min], [alpha3_min], [beta1_min],
            [beta2_min], [beta3_min], [delta1_std], [delta2_std], [delta3_std], [theta1_std], [theta2_std],
            [theta3_std], [alpha1_std], [alpha2_std], [alpha3_std], [beta1_std], [beta2_std], [beta3_std],
            [delta1_prct], [delta2_prct], [delta3_prct], [theta1_prct], [theta2_prct], [theta3_prct], [alpha1_prct],
            [alpha2_prct], [alpha3_prct], [beta1_prct], [beta2_prct], [beta3_prct], [delthe1_mean], [delthe2_mean],
            [delthe3_mean], [delthe4_mean], [delalph1_mean], [delalph2_mean], [delalph3_mean], [delalph4_mean],
            [thealph1_mean], [thealph2_mean], [thealph3_mean], [thealph4_mean], [delthe1_min], [delthe2_min],
            [delthe3_min], [delalph1_min], [delalph2_min], [delalph3_min], [thealph1_min], [thealph2_min],
            [thealph3_min], [delthe1_std], [delthe2_std], [delthe3_std], [delalph1_std], [delalph2_std], [delalph3_std],
            [thealph1_std], [thealph2_std], [thealph3_std], [delthe1_prct], [delthe2_prct], [delthe3_prct],
            [delalph1_prct], [delalph2_prct], [delalph3_prct], [thealph1_prct], [thealph2_prct], [thealph3_prct],
            [kurtdelta1], [kurtdelta2], [kurtdelta3], [kurttheta1], [kurttheta2], [kurttheta3], [kurtalpha1],
            [kurtalpha2], [kurtalpha3], [kurtbeta1], [kurtbeta2], [kurtbeta3]))
    features2 = matlabarray(
        cat([delta4_min], [theta4_min], [alpha4_min], [beta4_min], [delta4_std], [theta4_std], [alpha4_std],
            [beta4_std], [delta4_prct], [theta4_prct], [alpha4_prct], [beta4_prct], [delthe4_min], [delalph4_min],
            [thealph4_min], [delthe4_std], [delalph4_std], [thealph4_std], [delthe4_prct], [delalph4_prct],
            [thealph4_prct]))

    # kurtbeta4];
    features = matlabarray(cat(features1, features1a, features1aa, features1b, features2))
    # features=matlabarray(cat([features1],[features2]))

    return features



In [ ]:
#### Full pipeline ###
def Step1_computeFeaturesFromRawData(fileName0=None, *args, **kwargs):    
    fp = fileName0
    with load_mat(fp):
        matfile = load(fp)
        Fs = cellarray(matfile['Fs'])
        start_time = cellarray(matfile['start_time'])
        # channels = cellarray((51,1))
        channels = cellarray(matfile['channels'])
        data = cellarray(matfile['data'])

    # TODO: Resample to 200 Hz!!! %
    # data = np.resample(data, 200, Fs)
    # data = data.T
    # Fs = 200


    print('targetFolder', targetFolder)

    def strrep_s(a, x, y):
        return a.replace(x, y)

    strrep = np.vectorize(strrep_s)

    print('channels', channels)
    print(size(channels, 1))
    labels = copy(channels)

    for i in arange(1, size(channels, 1)).reshape(-1):
        #     print(i)
        x = strrep(channels[i, :], ' ', '')
        labels[i] = lower(x)


    disp('load 0')
    disp('compute 1')
    # pd=copy(pwd)
    movingwin = matlabarray(cat(2, 2))

    # dictobj
    params = Dict2Obj({'default': 1})

    params.pad = copy(0)
    params.fpass = copy(cat(0.5, 20))
    params.err = copy(0)
    params.trialave = copy(0)
    params.tapers = copy(cat(2, 3))
    params.Fs = copy(200)


    COI = cell(4, )
    COI[1] = cellarray([['fp1', 'f7'], ['f7', 't3'], ['t3', 't5'], ['t5', 'o1']])
    COI[2] = cellarray([['fp1', 'f3'], ['f3', 'c3'], ['c3', 'p3'], ['p3', 'o1']])
    COI[3] = cellarray([['fp2', 'f4'], ['f4', 'c4'], ['c4', 'p4'], ['p4', 'o2']])
    COI[4] = cellarray([['fp2', 'f8'], ['f8', 't4'], ['t4', 't6'], ['t6', 'o2']])

    print('COI', COI)

    ROInickname = cellarray(['LL', 'LP', 'RP', 'RL'])
    print(ROInickname)

    # test
    def strcmp_s(x, y):
        return x == y
    strcmp = np.vectorize(strcmp_s)

    dataB = cell(4, 4)
    for kk in arange(1, 4).reshape(-1):
        coi = COI[kk]

        for k in arange(1, size(coi, 1)).reshape(-1):
            print('kk', kk, 'k', k, coi[k, 1], coi[k, 2])

            c1 = find(strcmp(labels, coi[k, 1]))
            c2 = find(strcmp(labels, coi[k, 2]))
            c1 = c1.item()
            c2 = c2.item()
            print('items', c1, c2)
            dataB[kk, k] = np.array((data[c1, :] - data[c2, :]).tolist())

    d = {}
    d['dataB'] = dataB
    outp = catstr(targetFolder, '/strcmp_', fileName0)
    save(outp, d)

    # compute spectrogram for each channel
    spect = cell(4, 4)
    disp('compute 2', 'compute spectrogram for each channel')
    print('params', params.Fs, movingwin)

    for kk in arange(1, 4).reshape(-1):
        for k in arange(1, 4).reshape(-1):
            print('kk', kk, 'k', k)
            s = dataB[kk, k]
            params.Fs = copy(Fs).item()

            S, stimes, sfreqs = mt_spectrogram(s, Fs)

            spect[kk, k] = np.array(S.T)
            # Average over this region
    d = {}
    d['dataB'] = dataB
    outp = catstr(targetFolder, '/compute2_', fileName0)
    save(outp, d)

    # compute regional averages
    disp('compute 3', 'compute regional averages')
    R = cell(4, 1)
    for kk in arange(1, 4).reshape(-1):
        # print('kk',kk,'size',size(spect[kk,1-1]),spect[kk,1-1].shape,tuple(size(spect[kk,1-1])))
        T = np.zeros(size(spect[kk, 1]))
        print('T', T)
        # T = zeros(sizespect[kk, 1 - 1].shape[0])
        print('T', T.shape)
        for k in arange(1, 4).reshape(-1):
            print('kk', kk, 'k', k)

            T = T + spect[kk, k]
        R[kk] = T / 4

    print('save 0')
    d = {}
    d['dataB'] = dataB
    d['sfreqs'] = sfreqs
    d['stimes'] = stimes
    d['Fs'] = Fs
    d['R'] = R
    outp = catstr(targetFolder, '/spect_', fileName0)
    save(outp, d)  # 7.3 by default

    # TODO: test spect_

    #### 1.2 Compute features ####
    disp('compute 4', 'compute features')
    Fs = Fs.item()
    epoch_length = dot(Fs, 2)
    window_length = dot(cat(14, 10, 6, 2), Fs)

    # filtering of data
    fnyq = Fs / 2  # nyquist frequency
    fc_high = 50
    fc_low = 0.5
    #  cutoff frequencies

    b, a = butter(6, cat(fc_low / fnyq, fc_high / fnyq), 'bandpass')

    for kk in arange(1, 4).reshape(-1):
        for k in arange(1, 4).reshape(-1):
            print('kk', kk, 'k', k)
            temp = dataB[kk, k]

            dataB[kk, k] = filtfilt(b, a, temp, padtype='odd', padlen=3 * (np.amax((len(a), len(b))) - 1))

    disp('save 1', outp)
    outp = catstr(targetFolder, '/filtered_data_', fileName0)
    d = {}
    d['dataB'] = dataB
    d['Fs'] = cellarray(Fs)
    # save(outp, dataB, Fs)
    save(outp, d)
    # TODO: Test_filt

    # ------

    disp('compute 5')
    spectwindow_length = floor(window_length / Fs / 2)

    # fix: per jingjing
    # nr_epochs=floor(length(dataB[1,1](arange(window_length[1] - epoch_length,end()))) / epoch_length)
    # N=length(dataB[1,1])
    # N = length(dataB[1-1, 1-1])
    # nr_epochs=floor((N - (window_length[1] - epoch_length) - Fs) / epoch_length)
    # nr_features=144

    N = length(dataB[1, 1])
    nr_epochs = floor((N - (window_length[1] - epoch_length) - Fs) / epoch_length)
    nr_features = 144

    # features=zeros(4,nr_features,nr_epochs)
    features = cell(4, nr_features, nr_epochs)
    # tic
    for kk in arange(1, 4).reshape(-1):
        spect = R[kk, 1]

        data = dataB[kk, :]
        print('kk', kk)

        i = nr_epochs
        features[kk, :, i] = get_features(data, int(i.item()), window_length, spect, sfreqs, spectwindow_length, Fs)

        # n_jobs = 1  # number of cpus for parallel computing, -1 is all cpus
        n_jobs = -1
        verbose = True  # verbosity in parallel computing
        res = Parallel(n_jobs=n_jobs, verbose=verbose)(
            # delayed(compute_spec_each_seg)(eeg[window_start[wi]:window_start[wi] + window_length, :], NW, Fs) for wi in range(window_num)
            delayed(get_features)(data, int(i.item()), window_length, spect, sfreqs, spectwindow_length, Fs) for i in
            arange(1, nr_epochs).reshape(-1)
        )

        print(len(res))
        for i, rr in enumerate(res):
            # rr[0][freq_good_ids]
            features[kk, :, i] = rr[0][i]

    # features=reshape(features,cat(dot(4,nr_features),length(features)))
    features = features.reshape(cat(dot(4, nr_features), length(features)))
    features = features.T

    outp = catstr(targetFolder, '/features_', fileName0)
    disp('save 2')
    # save(cat(targetFolder,'/features_',fileName0),'features','window_length')
    d = {}
    d['features'] = features
    d['window_length'] = window_length
    # save(outp, features, window_length)
    save(outp, d)

    status = 0
    return status


# test
if __name__ == '__main__':
    pass
    # filep=
    filep=''
    
    targetFolder = './'
    Step1_computeFeaturesFromRawData(fileName0=filep)

In [22]:
class load_mat():
    def __init__(self, filename, mode='r'):
        self.filename = filename
        self.mode = mode

    def __enter__(self):
        print('open')
        # self.open_file = open(self.filename, self.mode)
        return 0

    def __exit__(self, *args):
        # self.open_file.close()
        print('close')

def Step1_computeFeaturesFromRawData_LoadData_ComputeSpectrogram(fileName0=None,*args,**kwargs):
#def Step1_computeFeaturesFromRawData_LoadData_ComputeSpectrogram(fileName0=None,*args,**kwargs):
#def Step1_computeFeaturesFromRawData(fileName0=None,*args,**kwargs):
    dataPath = wdir + '/Data/'

    print('dataPath',dataPath)
    
    targetFolder = catstr(wdir, '/Output/')
    import pandas as pd

    #### 1.1 Compute spectrograms ####
    
    fp=dataPath +fileName0
    with load_mat(fp):
        matfile = load(fp)
        Fs=cellarray(matfile['Fs'])
        start_time = cellarray(matfile['start_time'])
        # channels = cellarray((51,1))
        channels=cellarray(matfile['channels'])
        data = cellarray(matfile['data'])

    print('targetFolder',targetFolder)

    def strrep_s(a,x,y):
        return a.replace(x,y)

    strrep=np.vectorize(strrep_s)


    print('channels',channels)
    print(size(channels,1))
    # df_s0=pd.DataFrame()
    labels=copy(channels)
    print(labels.shape)
    # for i in arange(1-1,size(channels,1)-1).reshape(-1):
    for i in arange(1, size(channels, 1)).reshape(-1):
        x=strrep(channels[i,:],' ','')
        labels[i] = lower(x)

    if(1):
        print('labels',labels)
        for e in labels:
            print(e)


    disp('load 0')
    disp('compute 1')
    # pd=copy(pwd)
    movingwin=matlabarray(cat(2,2))

    # dictobj
    params=Dict2Obj({'default':1})

    params.pad = copy(0)
    params.fpass = copy(cat(0.5,20))
    params.err = copy(0)
    params.trialave = copy(0)
    params.tapers = copy(cat(2,3))
    params.Fs = copy(200)

    # spect=cell(4,4)
    # COI=numpy.empty((4, ), dtype=object)
    COI=cell(4,)
    COI[1]=cellarray([['fp1','f7'],['f7','t3'],['t3','t5'],['t5','o1']])
    COI[2]=cellarray([['fp1','f3'],['f3','c3'],['c3','p3'],['p3','o1']])
    COI[3]=cellarray([['fp2','f4'],['f4','c4'],['c4','p4'],['p4','o2']])
    COI[4]=cellarray([['fp2','f8'],['f8','t4'],['t4','t6'],['t6','o2']])

    print('COI',COI)
    
    ROInickname=cellarray(['LL','LP','RP','RL'])
    print(ROInickname)


    # test
    def strcmp_s(x,y):
        return x == y
        
    strcmp=np.vectorize(strcmp_s)

    dataB=cell(4,4)
    for kk in arange(1 , 4 ).reshape(-1):
        # print('kk',kk)
        coi=COI[kk]
        print('coi',coi,size(coi,1),size(coi,2),size(coi))

        # for k in arange(1-1,size(coi,1-1)-1).reshape(-1):
        for k in arange(1 , size(coi, 1) ).reshape(-1):
            print('kk',kk,'k',k , coi[k,1],coi[k,2])

            c1=find(strcmp(labels,coi[k,1]))
            c2=find(strcmp(labels,coi[k,2]))
            c1=c1.item()
            c2 = c2.item()
            dataB[kk, k] = np.array((data[c1, :] - data[c2, :]).tolist())
            
            

    spect = cell(4,4)
    disp('compute 2','compute spectrogram for each channel')
    print('params',params.Fs,movingwin)

    for kk in arange(1,4).reshape(-1):
        for k in arange(1,4).reshape(-1):
            print('kk',kk,'k',k)
            s=dataB[kk,k]
            params.Fs = copy(Fs).item()


            # S,stimes,sfreqs=mtspecgram_mbw(s,params,movingwin,1,nargout=3)

            # S, stimes, sfreqs = mtspecgram(s, params, movingwin)
            S, stimes, sfreqs = mt_spectrogram(s, Fs)

            print(S,S.T)
            # spect[kk,k]=S.T
            spect[kk, k] = np.array(S.T)
        # Average over this region
    d={}
    d['dataB']=dataB
    outp=catstr(targetFolder, '/compute2_', fileName0)
    save(outp, d)



    # compute regional averages
    disp('compute 3','compute regional averages')
    # R=cell(4,1)
    R=cell(4,1)
    for kk in arange(1,4).reshape(-1):
        # print('kk',kk,'size',size(spect[kk,1-1]),spect[kk,1-1].shape,tuple(size(spect[kk,1-1])))
        T=np.zeros(size(spect[kk,1]))
        print('T',T)
        # T = zeros(sizespect[kk, 1 - 1].shape[0])
        print('T',T.shape)
        for k in arange(1,4).reshape(-1):
            print('kk',kk,'k',k)

            T=T + spect[kk,k]
        R[kk]=T / 4
    